# Python Lab 5: BER Performance Simulation

**EE 451: Communications Systems**

**Name:** _________________________ **Date:** _____________

---

## Objectives

1. Simulate bit error rate (BER) for various modulation schemes
2. Compare simulated BER with theoretical curves
3. Understand the relationship between Eb/N0 and BER
4. Analyze the trade-off between bandwidth efficiency and power efficiency
5. Implement symbol detection and error counting

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special

plt.rcParams['figure.figsize'] = (10, 8)
np.random.seed(42)
print("Setup complete!")

## Part 1: BPSK BER Simulation (25 points)

BPSK maps bits to antipodal symbols: 0 → -1, 1 → +1

### Theoretical BER

For BPSK in AWGN:
$$P_b = Q\left(\sqrt{\frac{2E_b}{N_0}}\right) = \frac{1}{2}\text{erfc}\left(\sqrt{\frac{E_b}{N_0}}\right)$$

### Task 1.1: Implement BPSK Simulation

In [ ]:
def simulate_bpsk_ber(eb_n0_db, num_bits=100000):
    """
    Simulate BPSK transmission and calculate BER.
    
    Parameters:
        eb_n0_db: Eb/N0 in dB
        num_bits: Number of bits to simulate
    
    Returns:
        ber: Bit error rate
    """
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # TODO: Map bits to BPSK symbols (0 -> -1, 1 -> +1)
    symbols = 2 * bits - 1  # BPSK mapping
    
    # Convert Eb/N0 from dB to linear
    eb_n0_linear = 10**(eb_n0_db / 10)
    
    # For BPSK with unit energy symbols: Es = Eb = 1
    # Noise variance = N0/2 = 1/(2*Eb/N0)
    noise_variance = 1 / (2 * eb_n0_linear)
    
    # TODO: Generate AWGN noise
    noise = np.sqrt(noise_variance) * np.random.randn(num_bits)
    
    # TODO: Add noise to symbols
    received = symbols + noise
    
    # TODO: Make hard decisions (threshold at 0)
    detected_bits = (received > 0).astype(int)
    
    # TODO: Count errors
    num_errors = np.sum(bits != detected_bits)
    ber = num_errors / num_bits
    
    return ber

# Test at Eb/N0 = 10 dB
test_ber = simulate_bpsk_ber(10)
print(f"BPSK BER at Eb/N0 = 10 dB: {test_ber:.6f}")
print(f"Expected (theoretical): {0.5 * special.erfc(np.sqrt(10**(10/10))):.6f}")

### Task 1.2: Generate BER Curve

In [ ]:
# TODO: Simulate BER for range of Eb/N0 values
eb_n0_db_range = np.arange(0, 12, 1)
simulated_ber_bpsk = []

print("Simulating BPSK BER...")
for eb_n0 in eb_n0_db_range:
    ber = simulate_bpsk_ber(eb_n0, num_bits=500000)
    simulated_ber_bpsk.append(ber)
    print(f"  Eb/N0 = {eb_n0:2d} dB: BER = {ber:.2e}")

# TODO: Calculate theoretical BER
eb_n0_linear = 10**(eb_n0_db_range / 10)
theoretical_ber_bpsk = 0.5 * special.erfc(np.sqrt(eb_n0_linear))

# Plot comparison
plt.figure(figsize=(10, 7))
plt.semilogy(eb_n0_db_range, simulated_ber_bpsk, 'bo', markersize=8, label='Simulated')
plt.semilogy(eb_n0_db_range, theoretical_ber_bpsk, 'b-', linewidth=2, label='Theoretical')
plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('BPSK BER: Simulated vs Theoretical')
plt.legend()
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-6, 1)
plt.show()

### Question 1.1 (10 points)

a) What Eb/N0 is required to achieve BER = 10⁻³ for BPSK?

b) Why do we use Eb/N0 instead of SNR when comparing modulation schemes?

c) At low Eb/N0 values, the simulated BER matches theory well. Why might they diverge at very high Eb/N0 values?

**Your Answer:**

*[Write your answer here]*

## Part 2: QPSK BER Simulation (20 points)

QPSK encodes 2 bits per symbol. With Gray coding, each symbol error typically causes only 1 bit error.

### Task 2.1: Implement QPSK Simulation

In [ ]:
def simulate_qpsk_ber(eb_n0_db, num_bits=100000):
    """
    Simulate QPSK transmission and calculate BER.
    
    Parameters:
        eb_n0_db: Eb/N0 in dB
        num_bits: Number of bits to simulate (must be even)
    
    Returns:
        ber: Bit error rate
    """
    # Ensure even number of bits
    num_bits = num_bits - (num_bits % 2)
    num_symbols = num_bits // 2
    
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # TODO: Map bit pairs to QPSK symbols (Gray coded)
    # I component from odd bits, Q component from even bits
    i_bits = bits[0::2]  # First bit of each pair
    q_bits = bits[1::2]  # Second bit of each pair
    
    # Map: 0 -> -1, 1 -> +1 for each component
    i_symbols = 2 * i_bits - 1
    q_symbols = 2 * q_bits - 1
    
    # Normalize to unit average energy per symbol
    symbols = (i_symbols + 1j * q_symbols) / np.sqrt(2)
    
    # Convert Eb/N0 to Es/N0
    # For QPSK: Es = 2*Eb (2 bits per symbol)
    eb_n0_linear = 10**(eb_n0_db / 10)
    es_n0_linear = 2 * eb_n0_linear
    
    # Noise variance per dimension = N0/2 = Es/(2*Es/N0)
    noise_variance = 1 / (2 * es_n0_linear)
    
    # TODO: Generate complex AWGN
    noise = np.sqrt(noise_variance) * (np.random.randn(num_symbols) + 
                                        1j * np.random.randn(num_symbols))
    
    # TODO: Add noise
    received = symbols + noise
    
    # TODO: Make hard decisions on I and Q separately
    detected_i_bits = (received.real > 0).astype(int)
    detected_q_bits = (received.imag > 0).astype(int)
    
    # Reconstruct bit stream
    detected_bits = np.zeros(num_bits, dtype=int)
    detected_bits[0::2] = detected_i_bits
    detected_bits[1::2] = detected_q_bits
    
    # Count errors
    num_errors = np.sum(bits != detected_bits)
    ber = num_errors / num_bits
    
    return ber

# Test at Eb/N0 = 10 dB
test_ber = simulate_qpsk_ber(10)
print(f"QPSK BER at Eb/N0 = 10 dB: {test_ber:.6f}")
print(f"Expected (same as BPSK): {0.5 * special.erfc(np.sqrt(10**(10/10))):.6f}")

### Task 2.2: Compare BPSK and QPSK

In [ ]:
# Simulate QPSK
simulated_ber_qpsk = []

print("Simulating QPSK BER...")
for eb_n0 in eb_n0_db_range:
    ber = simulate_qpsk_ber(eb_n0, num_bits=500000)
    simulated_ber_qpsk.append(ber)
    print(f"  Eb/N0 = {eb_n0:2d} dB: BER = {ber:.2e}")

# Plot comparison
plt.figure(figsize=(10, 7))
plt.semilogy(eb_n0_db_range, simulated_ber_bpsk, 'bo', markersize=8, label='BPSK (simulated)')
plt.semilogy(eb_n0_db_range, simulated_ber_qpsk, 'rs', markersize=8, label='QPSK (simulated)')
plt.semilogy(eb_n0_db_range, theoretical_ber_bpsk, 'k-', linewidth=2, label='Theoretical')
plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('BPSK vs QPSK BER Performance')
plt.legend()
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-6, 1)
plt.show()

print("\nKey observation: BPSK and QPSK have the same BER vs Eb/N0!")
print("But QPSK is 2x more bandwidth efficient (2 bits/symbol vs 1 bit/symbol)")

### Question 2.1 (10 points)

a) Why do BPSK and QPSK have the same BER vs Eb/N0 performance?

b) If they have the same BER performance, why would you ever use QPSK instead of BPSK?

**Your Answer:**

*[Write your answer here]*

## Part 3: 16-QAM BER Simulation (25 points)

16-QAM encodes 4 bits per symbol using 16 constellation points.

### Task 3.1: Implement 16-QAM Simulation

In [ ]:
def simulate_16qam_ber(eb_n0_db, num_bits=100000):
    """
    Simulate 16-QAM transmission and calculate BER.
    
    Parameters:
        eb_n0_db: Eb/N0 in dB
        num_bits: Number of bits to simulate (must be multiple of 4)
    
    Returns:
        ber: Bit error rate
    """
    # Ensure multiple of 4 bits
    num_bits = num_bits - (num_bits % 4)
    num_symbols = num_bits // 4
    
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # TODO: Create 16-QAM constellation (Gray coded)
    # Levels: -3, -1, +1, +3
    # Gray coded mapping for 2 bits -> level
    gray_map = {(0, 0): -3, (0, 1): -1, (1, 1): +1, (1, 0): +3}
    
    # Map 4 bits to symbol: first 2 bits -> I, last 2 bits -> Q
    symbols = np.zeros(num_symbols, dtype=complex)
    for i in range(num_symbols):
        b = bits[4*i:4*i+4]
        i_level = gray_map[(b[0], b[1])]
        q_level = gray_map[(b[2], b[3])]
        symbols[i] = i_level + 1j * q_level
    
    # Normalize to unit average energy
    avg_power = np.mean(np.abs(symbols)**2)
    symbols = symbols / np.sqrt(avg_power)
    
    # Convert Eb/N0 to Es/N0
    # For 16-QAM: Es = 4*Eb (4 bits per symbol)
    eb_n0_linear = 10**(eb_n0_db / 10)
    es_n0_linear = 4 * eb_n0_linear
    
    # Noise variance per dimension
    noise_variance = 1 / (2 * es_n0_linear)
    
    # Generate and add noise
    noise = np.sqrt(noise_variance) * (np.random.randn(num_symbols) + 
                                        1j * np.random.randn(num_symbols))
    received = symbols + noise
    
    # TODO: Detect symbols (hard decision on each component)
    # Inverse Gray map: level -> (bit0, bit1)
    def detect_level(val):
        """Map received value to Gray-coded bits."""
        # Scale back to original levels
        val_scaled = val * np.sqrt(avg_power)
        # Decision boundaries at -2, 0, +2
        if val_scaled < -2:
            return (0, 0)  # -3
        elif val_scaled < 0:
            return (0, 1)  # -1
        elif val_scaled < 2:
            return (1, 1)  # +1
        else:
            return (1, 0)  # +3
    
    # Detect all symbols
    detected_bits = np.zeros(num_bits, dtype=int)
    for i in range(num_symbols):
        i_bits = detect_level(received[i].real)
        q_bits = detect_level(received[i].imag)
        detected_bits[4*i:4*i+4] = [i_bits[0], i_bits[1], q_bits[0], q_bits[1]]
    
    # Count errors
    num_errors = np.sum(bits != detected_bits)
    ber = num_errors / num_bits
    
    return ber

# Test at Eb/N0 = 14 dB
test_ber = simulate_16qam_ber(14)
print(f"16-QAM BER at Eb/N0 = 14 dB: {test_ber:.6f}")

### Task 3.2: Generate Multi-Scheme Comparison

In [ ]:
# Extended range for 16-QAM
eb_n0_db_extended = np.arange(0, 16, 1)

# Simulate all schemes
print("Simulating BER for all modulation schemes...")

ber_bpsk = []
ber_qpsk = []
ber_16qam = []

for eb_n0 in eb_n0_db_extended:
    ber_bpsk.append(simulate_bpsk_ber(eb_n0, num_bits=200000))
    ber_qpsk.append(simulate_qpsk_ber(eb_n0, num_bits=200000))
    ber_16qam.append(simulate_16qam_ber(eb_n0, num_bits=200000))
    print(f"  Eb/N0 = {eb_n0:2d} dB: BPSK={ber_bpsk[-1]:.2e}, QPSK={ber_qpsk[-1]:.2e}, 16-QAM={ber_16qam[-1]:.2e}")

# Theoretical curves
eb_n0_linear = 10**(eb_n0_db_extended / 10)
theory_bpsk = 0.5 * special.erfc(np.sqrt(eb_n0_linear))

# 16-QAM approximation: Pb ≈ (3/8)*erfc(sqrt(2*Eb/5*N0))
theory_16qam = (3/8) * special.erfc(np.sqrt(0.4 * eb_n0_linear))

# Plot
plt.figure(figsize=(12, 8))
plt.semilogy(eb_n0_db_extended, ber_bpsk, 'bo', markersize=6, label='BPSK (sim)')
plt.semilogy(eb_n0_db_extended, ber_qpsk, 'gs', markersize=6, label='QPSK (sim)')
plt.semilogy(eb_n0_db_extended, ber_16qam, 'r^', markersize=6, label='16-QAM (sim)')
plt.semilogy(eb_n0_db_extended, theory_bpsk, 'b-', linewidth=2, label='BPSK/QPSK (theory)')
plt.semilogy(eb_n0_db_extended, theory_16qam, 'r--', linewidth=2, label='16-QAM (approx)')

plt.xlabel('Eb/N0 (dB)', fontsize=12)
plt.ylabel('Bit Error Rate (BER)', fontsize=12)
plt.title('BER Comparison: BPSK, QPSK, and 16-QAM', fontsize=14)
plt.legend(loc='lower left', fontsize=10)
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-6, 1)
plt.xlim(0, 15)

# Add reference lines
plt.axhline(y=1e-3, color='gray', linestyle=':', alpha=0.7)
plt.text(0.5, 1.5e-3, 'BER = 10⁻³', fontsize=9)

plt.tight_layout()
plt.show()

### Question 3.1 (15 points)

a) At BER = 10⁻³, how much more Eb/N0 does 16-QAM require compared to BPSK?

b) 16-QAM sends 4 bits per symbol while BPSK sends 1 bit per symbol. Given this 4x bandwidth efficiency improvement, is the Eb/N0 penalty worth it? Explain.

c) For a mobile phone system with limited battery power but ample bandwidth, would you choose BPSK or 16-QAM? What about for satellite TV with limited bandwidth?

**Your Answer:**

*[Write your answer here]*

## Part 4: Eb/N0 vs SNR Conversion (15 points)

### Task 4.1: Understand the Relationship

In [ ]:
# Relationship: SNR = (Eb/N0) * (Rb/B)
# For ideal Nyquist signaling: B = Rs (symbol rate)
# Rb = Rs * log2(M) where M = constellation size
#
# Therefore: SNR = (Eb/N0) * log2(M)

def eb_n0_to_snr(eb_n0_db, bits_per_symbol):
    """Convert Eb/N0 to SNR."""
    return eb_n0_db + 10 * np.log10(bits_per_symbol)

def snr_to_eb_n0(snr_db, bits_per_symbol):
    """Convert SNR to Eb/N0."""
    return snr_db - 10 * np.log10(bits_per_symbol)

# TODO: Create conversion table
print("Eb/N0 to SNR Conversion Table")
print("=" * 60)
print(f"{'Modulation':<12} {'Bits/Symbol':<12} {'Eb/N0 (dB)':<12} {'SNR (dB)':<12}")
print("-" * 60)

eb_n0_ref = 10  # Reference Eb/N0

modulations = [
    ('BPSK', 1),
    ('QPSK', 2),
    ('8-PSK', 3),
    ('16-QAM', 4),
    ('64-QAM', 6),
    ('256-QAM', 8)
]

for name, bits in modulations:
    snr = eb_n0_to_snr(eb_n0_ref, bits)
    print(f"{name:<12} {bits:<12} {eb_n0_ref:<12.1f} {snr:<12.1f}")

print("\nNote: Higher-order modulation requires higher SNR for same Eb/N0")

In [ ]:
# TODO: Plot BER vs SNR (instead of Eb/N0)
snr_bpsk = [eb_n0_to_snr(x, 1) for x in eb_n0_db_extended]
snr_qpsk = [eb_n0_to_snr(x, 2) for x in eb_n0_db_extended]
snr_16qam = [eb_n0_to_snr(x, 4) for x in eb_n0_db_extended]

plt.figure(figsize=(12, 8))

# Plot vs Eb/N0
plt.subplot(1, 2, 1)
plt.semilogy(eb_n0_db_extended, ber_bpsk, 'b-o', label='BPSK')
plt.semilogy(eb_n0_db_extended, ber_qpsk, 'g-s', label='QPSK')
plt.semilogy(eb_n0_db_extended, ber_16qam, 'r-^', label='16-QAM')
plt.xlabel('Eb/N0 (dB)')
plt.ylabel('BER')
plt.title('BER vs Eb/N0 (Power Efficiency View)')
plt.legend()
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-5, 1)

# Plot vs SNR
plt.subplot(1, 2, 2)
plt.semilogy(snr_bpsk, ber_bpsk, 'b-o', label='BPSK (1 bit/sym)')
plt.semilogy(snr_qpsk, ber_qpsk, 'g-s', label='QPSK (2 bits/sym)')
plt.semilogy(snr_16qam, ber_16qam, 'r-^', label='16-QAM (4 bits/sym)')
plt.xlabel('SNR (dB)')
plt.ylabel('BER')
plt.title('BER vs SNR (Bandwidth Efficiency View)')
plt.legend()
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-5, 1)

plt.tight_layout()
plt.show()

print("Left plot shows power efficiency: BPSK/QPSK are best")
print("Right plot shows the SNR required for each scheme at each spectral efficiency")

### Question 4.1 (15 points)

a) A WiFi system operates at SNR = 25 dB. What is the Eb/N0 for:
   - QPSK?
   - 64-QAM?

b) Explain why BER vs SNR curves cross over (16-QAM can have lower BER than BPSK at same SNR) while BER vs Eb/N0 curves never cross.

c) The Shannon capacity formula is C = B·log₂(1 + SNR). How does this relate to the choice between modulation schemes?

**Your Answer:**

*[Write your answer here]*

## Part 5: Coding Gain Preview (15 points)

Error correcting codes can improve BER at the cost of reduced data rate.

### Task 5.1: Simulate Simple Repetition Coding

In [ ]:
def simulate_repetition_code(eb_n0_db, repetitions=3, num_bits=50000):
    """
    Simulate BPSK with repetition coding.
    
    Each bit is repeated 'repetitions' times. Receiver uses
    majority voting to decode.
    """
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # TODO: Repeat each bit
    coded_bits = np.repeat(bits, repetitions)
    
    # Map to BPSK symbols
    symbols = 2 * coded_bits - 1
    
    # Account for rate loss: Eb_coded = Eb_uncoded / rate
    # Rate = 1/repetitions, so coded Eb/N0 is lower
    eb_n0_coded_db = eb_n0_db - 10 * np.log10(repetitions)
    eb_n0_linear = 10**(eb_n0_coded_db / 10)
    noise_variance = 1 / (2 * eb_n0_linear)
    
    # Add noise
    noise = np.sqrt(noise_variance) * np.random.randn(len(symbols))
    received = symbols + noise
    
    # TODO: Soft combining - sum repeated symbols
    received_reshaped = received.reshape(num_bits, repetitions)
    combined = np.sum(received_reshaped, axis=1)
    
    # Hard decision on combined signal
    detected_bits = (combined > 0).astype(int)
    
    # Count errors
    num_errors = np.sum(bits != detected_bits)
    ber = num_errors / num_bits
    
    return ber

# Compare uncoded vs repetition coded
print("Simulating repetition coding...")

ber_uncoded = []
ber_rep3 = []
ber_rep5 = []

for eb_n0 in eb_n0_db_extended:
    ber_uncoded.append(simulate_bpsk_ber(eb_n0, num_bits=100000))
    ber_rep3.append(simulate_repetition_code(eb_n0, repetitions=3, num_bits=100000))
    ber_rep5.append(simulate_repetition_code(eb_n0, repetitions=5, num_bits=100000))

# Plot
plt.figure(figsize=(10, 7))
plt.semilogy(eb_n0_db_extended, ber_uncoded, 'b-o', label='Uncoded BPSK')
plt.semilogy(eb_n0_db_extended, ber_rep3, 'g-s', label='Rep-3 (rate 1/3)')
plt.semilogy(eb_n0_db_extended, ber_rep5, 'r-^', label='Rep-5 (rate 1/5)')

plt.xlabel('Eb/N0 (dB) - per information bit')
plt.ylabel('Bit Error Rate (BER)')
plt.title('Effect of Repetition Coding on BER')
plt.legend()
plt.grid(True, which='both', alpha=0.3)
plt.ylim(1e-6, 1)
plt.show()

print("\nNote: Coding provides gain at high Eb/N0 but costs bandwidth (lower rate)")

### Question 5.1 (15 points)

a) At BER = 10⁻⁴, approximately how much coding gain (in dB) does the rate-1/3 repetition code provide over uncoded BPSK?

b) The rate-1/3 code sends each bit 3 times, reducing throughput by 3x. Is this trade-off worthwhile? When might it be acceptable?

c) Modern codes like Turbo codes and LDPC can achieve near-Shannon-limit performance. What theoretical limit does the Shannon capacity theorem set on the minimum Eb/N0 for error-free communication?

**Your Answer:**

*[Write your answer here]*

---

## Lab Summary

### Key Results

1. **BPSK and QPSK** have identical BER vs Eb/N0 performance
2. **16-QAM** requires ~5 dB more Eb/N0 than QPSK for same BER
3. **Trade-off**: Higher-order modulation = more bits/symbol but needs higher Eb/N0
4. **Eb/N0 vs SNR**: SNR = Eb/N0 + 10·log₁₀(bits per symbol)
5. **Coding gain**: Error correction improves BER but reduces throughput

### Important Equations

- **BPSK/QPSK BER**: $P_b = \frac{1}{2}\text{erfc}\left(\sqrt{E_b/N_0}\right)$
- **Eb/N0 to SNR**: $\text{SNR} = \frac{E_b}{N_0} \cdot \log_2(M)$
- **Shannon limit**: $E_b/N_0 > \ln(2) \approx -1.6$ dB for error-free communication

### Submission Checklist

- [ ] All TODO sections completed
- [ ] All questions answered with explanations
- [ ] All simulation plots generated
- [ ] Results verified against theoretical values
- [ ] Notebook saved and exported

---

*Submit to Brightspace by due date.*